In [ ]:
#Created By: Rowan Converse
#Date: 2023-03-07
#Purpose: Consensus analysis of crowdsourced Zooniverse labels of imagery from Bosque del Apache and Maxwell NWR, Ladd S Gordon Waterfowl Complex, Rio Grande State Park

In [ ]:
#Imports


In [ ]:
#Analysis annotations
dgc_path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/20230307_zooniverseanalysislabels.csv"
seagull_path = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/Dissertation/1_Chapter/consensus/data/20230307_zooniverseanalysislabels_seagull.csv"

#Refined annotations
ref_dgc = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_zooniverseconsensuslabels.csv"
ref_seagull = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_zooniverseconsensuslabels_seagull.csv"

In [ ]:
#Refined dataset statistics

In [ ]:
#Search for images that have an unexpected number of clusters and remove from analysis

In [ ]:
#INDIVIDUAL LABELER METRICS

#Score agreement with consensus class label

#Score %IOU agreement with consensus box 